In [1]:
# General
import os
import sys
import time
import shutil
import funcy as fp
import numpy as np
import pandas as pd
from functools import partial

# Verificação de tipos
from typing import List

# Visualization / Presentation
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

# Bibliotecas e Funções para NLP
import re
import fasttext
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# Treinamento e Avaliação de Modelos
from sklearn import metrics
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

# Algoritmos adicionais para a criação de modelos
from lightgbm.sklearn import LGBMModel
from catboost import CatBoostClassifier

# Rastreamento de experimentos e modelos
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature

# Recursos para visualização dos dados
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

# Carregar código personalizado disponível em ../src
sys.path.append(os.path.abspath(os.path.pardir))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.utils.experiments import set_dataset_split, compute_multiclass_classification_metrics

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

In [2]:
EXPERIMENT_NAME = '01_ShallowModels'
EXPERIMENT_RUN_NAME = f'Structuring'

mlflow_client = MlflowClient()

experiment = mlflow_client.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    mlflow_client.create_experiment(EXPERIMENT_NAME)
    experiment = mlflow_client.get_experiment_by_name(EXPERIMENT_NAME)

EXPERIMENT_ID = experiment.experiment_id
del experiment

# Importante: Este notebook está em estágio inicial de desenvolvimento

TODO (ou devaneios):
 - Criar mecanismo para preencher valores ausentes;
 - Normalizar dados para fazer experimentação com modelos além do Catboost;
 - Padronizar e criar wrapper para persistir pipeline e modelo do experimento;
 - Apresentar métricas de avaliação por algoritmo e por algoritmo + categoria;
 - Avaliar qual estratégia seguir para validação: 
   - Cross-validation orientada por tempo ou amostras aleatórias (provavelmente o segundo, considerando que o conjunto de teste já está separado para a avaliação final)
 - Experimentações diversas:
   - Avaliação de atributos;
   - Uso de embeddings para titulo, tags, titulo + tags;
   - Avaliar uso ou não de classes para balanceamento de categorias;
   - Avaliar estratégias para se lidar com produtos repetidos e com mudanças ao longo do tempo;
   - Analisar categorias com pior desempenho e verificar casos com erros;

# Classificação de Produtos em Categorias

Este notebook tem como propósito viabilizar os experimentos de criação de um classificador de produtos em categorias. Ainda que seja possível utilizar esse classificador para revisar produtos que possam estar categorizados de forma incorreta, é provável que exista um valor de utilização maior na possibilidade de classificar produtos recém cadastrados ou em processo de registro. Com isso, **não se deve esperar a disponibilidade de colunas relacionadas a busca ou às interações com os produtos**, como:
 - *query*
 - *search_page*
 - *position*
 - *view_counts*
 - *order_counts*
 
Uma **possibilidade** de incorporar algumas dessas informações, como visualizações e pedidos, seria a **de utilizar uma estatística (e.g., média ou mediana) de produtos similares** que já tinham sido cadastrados antes do momento da criação do produto em questão. Por exemplo, fazer a média dos 50 produtos com título e preço mais semelhantes. De início, esses campos e possibilidades não serão exploradas, considerando que **pode não existir volume suficiente para trazer informações representativas de todos os produtos**. Pelo mesmo motivo, provavelmente **não serão utilizadas estatísticas sobre vendedores**.

PS.: Se a qualidade do classificador não for satisfatória com as informações mínimas planejadas inicialmente, poderei apelar para as estratégias alternativas de incorporação dos demais campos. :0)

Além dos atributos que provavelmente não estarão disponíveis no momento de uso do classificador, é preciso as variações das características dos produtos. Conforme a [Análise Exploratória](02_Analise_Exploratoria.ipynb), **preço (*price*), peso (*weight*), pronta entrega (*express_delivery*) e quantidade mínima (*minimum_quantity*) variam ao longo do tempo para um mesmo produto**. Assim, é preciso tomar uma decisão sobre como lidar com a repetição dos produtos nos dados:
 - Lidar apenas com produtos únicos:
   - Descartar atributos que variam;
   - ~Usar valor mais recente de cada atributo~ (não é viável sem data da busca);
   - Calcular estatística sobre valores que variam;
 - Usar repetições dos produtos e  re-balancear o peso deles no treinamento do modelo; 
 
A utilização de produtos repetidos, com características que mudam com o tempo, pode ser interessante para trazer mais varidade de dados ao modelo, como uma aumentação natural. Caso seja preciso, pode-se tentar simular
 
Essas questões podem ser avaliadas de modo relativamente rápido entre as experimentações.




### Carregamento de Dados

Para trabalhar o problema de classificação de produtos em categorias, é preciso utilizar o conjunto de dados de treinamento, dividido em  [01_Estruturacao.ipynb](01_Estruturacao.ipynb). Ainda que 
, recuperar apenas as informações que podem ser us

In [3]:
columns_to_read = ['title', 'concatenated_tags', 'price', 'weight', 'express_delivery', 'minimum_quantity', 'category', 'creation_date']

frame = (pd
         .read_csv(os.path.join(settings.DATA_PATH, 'interim', 'training.csv'), usecols=columns_to_read)
         .drop_duplicates()  # Manter mais de uma ocorrência de produto apenas se existir variação nos dados
        )

In [4]:
cut_off_period = '2018-05'
split_frame = set_dataset_split(frame, cut_off_period)

training_frame = split_frame.loc[lambda f: f['group'] != 'test'].drop(columns=['group'])
validation_frame = split_frame.loc[lambda f: f['group'] == 'test'].drop(columns=['group'])

In [5]:
display_side_by_side([training_frame.head(2).T, validation_frame.head(2).T],
                     ['Treinamento', 'Validação'],
                     padding=50)

,0,1
title,Mandala Espírito Santo,Cartão de Visita
concatenated_tags,mandala mdf,cartao visita panfletos tag adesivos copos long drink canecas
creation_date,2015-11-14 19:42:12,2018-04-04 20:55:07
price,171.89,77.67
weight,1200.0,8.0
express_delivery,1,1
minimum_quantity,4,5
category,Decoração,Papel e Cia
period,2015-11,2018-04
,4,7


### Gerar Vetor de Características

Conforme analisado em [Analise_Textual](02.1_Analise_Textual.ipynb), a primeira estratégia a ser explorada para trabalar com texto será utilizar *embeddings*. Com isso, espera-se contornar eventuais problemas com palavras fora do vocabulário e reduzir a dimensionalidade dos dados de treinamento. Adicionalmente, pode-se utilizar as informações de similaridade para fazer aumentação de dados textuais ou recuperar itens semelhantes para derivar outros valores (e.g., média de popularidade ou peso de de produtos semelhantes).

In [6]:
ft_model = fasttext.load_model(os.path.join(settings.MODELS_PATH, 'cc.pt.300.bin'))

Criar codificador das categorias dos produtos.

In [7]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(training_frame.category)

y_training = label_encoder.transform(training_frame.category)
y_validation = label_encoder.transform(validation_frame.category)

Iniciar organização do processamento das caracerísticas

In [8]:
def create_feature_vector(base_frame: pd.DataFrame, feature_columns: List[str], columns_to_encode_as_embeddings: List[str]) -> np.array:
    "Função para processar gerar embeddings, criar processar demais features e criar vetor final para treino/inferência."
    embeddings_columns = []
    basic_columns = []

    for column in columns_to_encode_as_embeddings:
        embedding_column = np.stack(base_frame[column]
                                    .str
                                    .lower()
                                    .apply(ft_model.get_sentence_vector)
                                    .to_numpy(), axis=0)
        embeddings_columns.append(embedding_column)

    basic_columns = [np.expand_dims(base_frame[column].to_numpy(), axis=1) 
                     for column in feature_columns]

    return np.concatenate(embeddings_columns + basic_columns, axis=1)

feature_columns = ['price', 'weight', 'express_delivery']
columns_to_encode_as_embeddings = ['title']

X_training = create_feature_vector(training_frame, feature_columns, columns_to_encode_as_embeddings)
X_validation = create_feature_vector(validation_frame, feature_columns, columns_to_encode_as_embeddings)

print(f'X training: {X_training.shape} | X validation: {X_validation.shape}')

X training: (27130, 303) | X validation: (5599, 303)


Como há um desbalanceamento no número de registros de cada categoria, é interessante fazer um contra-balanceamento dos pesos para que exista uma proporcionalidade entre eles para o modelo. Assim, categorias com mais itens devem ter peso menor, enquanto categorias com menos itens possuem peso maior.

In [9]:
n_samples = len(training_frame)
n_classes = len(training_frame['category'].unique())

value_counts_frame = (training_frame
                      [['category']]
                      .assign(records=1)
                      .groupby(['category'])
                      .sum()
                      .reset_index()                      
                      .assign(weight=lambda f: f['records'].apply(lambda r: n_samples / (n_classes * r)))
                      .assign(label=lambda f: label_encoder.transform(f['category']))
                     )
display_side_by_side([value_counts_frame], ['Distribuição de Categorias e Pesos'])

class_weights = {item.label: item.weight for item in value_counts_frame[['label', 'weight']].itertuples(index=False)}

,category,records,weight,label
0,Bebê,5157,0.876802,0
1,Bijuterias e Jóias,675,6.698765,1
2,Decoração,6493,0.696391,2
3,Lembrancinhas,12234,0.369598,3
4,Outros,770,5.872294,4
5,Papel e Cia,1801,2.510642,5


A seguir, tem-se a estrutura de exploração de algoritmos e parâmetros que podem ser utilizados para criar modelos de classificação. Com o uso do MLFlow, cada experimento pode ser registrado para que o histórico sirva para avaliar as mudanças que foram mais eficazes.

In [10]:
num_classes = len(label_encoder.classes_)

models_parameters = {    
    'MLP': {'activation':'tanh', 'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'alpha': 0.001, 'early_stopping': True, 'hidden_layer_sizes':(200, 200)},
    'LGB': {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': num_classes, 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 0.5955392344062058, 
            'lambda_l2': 1.1394638571883556e-08, 'num_leaves': 35, 'feature_fraction': 0.7, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 5, 
            'num_iterations': 200, 'early_stopping_round': None, 'random_state': None, 'class_weight': 'balanced'},
    'CB': {'loss_function': 'MultiClass', 'classes_count': num_classes, 'iterations': 200, 'save_snapshot': False, 'verbose': False, 'class_weights': class_weights},
    'MultinomialNB': {'alpha': 0.125},
}

fit_parameters = {    
    'MLP': {},
    'LGB': {},
    'CB': {},
    'MultinomialNB': {},
}

models_to_train = {
    #'MLP': MLPClassifier,
    #'LGB': LGBMModel,
    'CB': CatBoostClassifier,
    #'MultinomialNB': MultinomialNB,
}

training_repetitions = 3
iterations_tracking = []
trained_models = []
validation_scores = []

with mlflow.start_run(run_name=EXPERIMENT_RUN_NAME, experiment_id=EXPERIMENT_ID) as main_run:
    # A cada execução limpa o diretório de artefatos para gravar novos, a serem salvos no MLflow
    if os.path.exists(settings.LOGS_ARTIFACTS_PATH):
        shutil.rmtree(settings.LOGS_ARTIFACTS_PATH)
    os.makedirs(settings.LOGS_ARTIFACTS_PATH) 

    mlflow.log_param('training_repetitions', training_repetitions)

    #mlflow.log_param('main_token_processors', ', '.join([item.__name__ for item in main_token_processors]))
    #mlflow.log_param('extra_token_processors', ', '.join([item.__name__ for item in extra_token_processors]))

    #mlflow.log_param('main_sentence_processors', ', '.join([item.__name__ for item in main_sentence_processors]))
    #mlflow.log_param('extra_sentence_processors', ', '.join([item.__name__ for item in extra_sentence_processors]))

    #simple_partial_clean_text_params, complex_partial_clean_text_params = format_nested_parameters(partial_clean_text.keywords, 'clean_text')
    #mlflow.log_params(simple_partial_clean_text_params)

    #simple_pipeline_params, complex_pipeline_params = format_nested_parameters(pipeline_parameters, 'pipeline')
    #mlflow.log_params(simple_pipeline_params)

    mlflow.log_param('X_training', X_training.shape)
    mlflow.log_param('X_validation', X_validation.shape)

    for ix in tqdm(range(training_repetitions)):
        iteration_index = np.arange(X_training.shape[0])
        np.random.shuffle(iteration_index)

        for model_name, model_class in models_to_train.items():

            with mlflow.start_run(run_name=f'01_{ix}_{EXPERIMENT_RUN_NAME}_{model_name}', experiment_id=EXPERIMENT_ID, nested=True) as nested_run:
                start_time = time.time()
                model = model_class(**models_parameters.get(model_name, {}))
                model.fit(X_training[iteration_index], y_training[iteration_index])
                mlflow.log_metric('training_time', time.time() - start_time)

                trained_models.append((model_name, ix, model)) 
                preds = model.predict(X_validation)

                eval_metrics = compute_multiclass_classification_metrics(y_validation, preds.round())
                iteration_tracking = {**{'Algorithm': model_name,
                                         'Iteration': ix}, 
                                      **eval_metrics}
                iterations_tracking.append(iteration_tracking)

                validation_scores.append((model_name, ix, eval_metrics['f1']))

                mlflow.log_param('model_name', model_name)
                mlflow.log_params(models_parameters.get(model_name, {}))
                mlflow.sklearn.log_model(model, "model")
                for key, value in eval_metrics.items():
                    mlflow.log_metric(key, np.mean(value))
                    
                # Save model
                signature = infer_signature(X_training, preds)
                mlflow.sklearn.log_model(model, model_name, signature=signature)

    # Métricas de avaliação individual
    evaluation_frame = pd.DataFrame(iterations_tracking)
    evaluation_frame.to_csv(os.path.join(settings.LOGS_ARTIFACTS_PATH, 'experiment_runs.csv'))
    evaluation_frame.to_html(os.path.join(settings.LOGS_ARTIFACTS_PATH, 'experiment_runs.html'))

    # Sumarização das métricas de várias execuções de um mesmo algoritmo
    evaluation_summary_frame = (evaluation_frame
                                [['Algorithm', 'acc', 'precision', 'recall', 'f1']]
                                .assign(**{item: lambda f: f[item].apply(np.mean) 
                                           for item in ['precision', 'recall', 'f1']})
                                .groupby('Algorithm')
                                .agg([np.mean, np.std])
                               )

    for item in evaluation_summary_frame.itertuples():
        mlflow.log_metric(item.Index, item._6)

    best_result_index = evaluation_summary_frame[('f1', 'mean')].argmax()
    for metric in ['acc', 'precision', 'recall', 'f1']:
        mlflow.log_metric(metric, evaluation_summary_frame.iloc[best_result_index][(metric, "mean")])

    evaluation_summary_frame.to_csv(os.path.join(settings.LOGS_ARTIFACTS_PATH, 'experiment_runs_summary.csv'))
    evaluation_summary_frame.to_html(os.path.join(settings.LOGS_ARTIFACTS_PATH, 'experiment_runs_summary.html'))

    #for param_name, param_value in {**complex_partial_clean_text_params, **complex_pipeline_params}.items():
    #    with open(f'{settings.LOGS_ARTIFACTS_PATH}/{param_name}.txt', 'w') as file:
    #        file.write(param_value)
            
    # Store the preprocessing resources
    #preprocessing_model_path = os.path.join(settings.LOGS_ARTIFACTS_PATH, 'preprocessing_model')
    #mlflow.pyfunc.save_model(path=preprocessing_model_path, python_model=preprocessing_wrapper)

    mlflow.log_artifact(settings.LOGS_ARTIFACTS_PATH)

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
evaluation_summary_frame

acc           precision              recall            \
               mean       std      mean       std      mean       std   
Algorithm                                                               
CB         0.839197  0.002728  0.773503  0.008579  0.773503  0.008579   

                 f1            
               mean       std  
Algorithm                      
CB         0.773503  0.008579